## Классические алгоритмы без ансамблирования
В этом ноутбуке вам нужно обучить модели на датасете классификации из предыдущего ноутбука и сравнить результаты. Вам будет предоставлен baseline, на основе которого вы будете доделывать предсказывающие модели. Оценка лабы будет зависеть от ROC-AUC на тестовых данных по следующим критериям:
\
AUC - на тестовых данных
- $AUC \leq 0.75$ - 0 баллов
- $0.75 < AUC \leq 0.76$ - 2 балла
- $0.76 < AUC \leq 0.77$ - 4 балла
- $0.77 < AUC \leq 0.78$ - 6 баллов
- $0.78 < AUC \leq 0.79$ - 8 баллов
- $AUC > 0.79$ - 10 баллов

\
В этой работе запрещено использовать ансамбли моделей (лес, бустинги и т.д.)!

In [ ]:
# Импорт необходимых библиотек
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, precision_score, recall_score, roc_curve, accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Загрузка данных из CSV файла
data = pd.read_csv('german.csv', sep=';')
print(data.head())  # Вывод первых 5 строк данных для проверки

# Определение признаков (X) и целевой переменной (y)
X = data.iloc[:, 1:].to_numpy()  # Все столбцы, кроме первого, как признаки
y = data.iloc[:, 0].to_numpy()    # Первый столбец как целевая переменная

# Разделение данных на обучающую и тестовую выборки (80% на обучение, 20% на тестирование)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

# Визуализация распределения классов в обучающих данных
plt.hist(y_train, bins=2, edgecolor='k')  # Гистограмма
plt.xticks([0, 1])  # Метки по оси X
plt.xlabel('Class (0: Non-Creditworthy, 1: Creditworthy)')  # Подпись оси X
plt.ylabel('Count')  # Подпись оси Y
plt.title('Distribution of Classes in Training Data')  # Заголовок графика
plt.show()  # Показ графика

In [ ]:
# Создание и обучение модели логистической регрессии
logistic_regression_model = LogisticRegression(max_iter=1000)  # Инициализация модели
logistic_regression_model.fit(X_train, y_train)  # Обучение модели

# Создание и обучение модели дерева решений
decision_tree_model = DecisionTreeClassifier(random_state=42)  # Инициализация модели
decision_tree_model.fit(X_train, y_train)  # Обучение модели

# Создание и обучение модели K-ближайших соседей
knn_model = KNeighborsClassifier()  # Инициализация модели
knn_model.fit(X_train, y_train)  # Обучение модели


In [ ]:
# Получение вероятностей предсказаний для тестовой выборки
y_prob_logistic = logistic_regression_model.predict_proba(X_test)[:, 1]
y_prob_decision_tree = decision_tree_model.predict_proba(X_test)[:, 1]
y_prob_knn = knn_model.predict_proba(X_test)[:, 1]

# Получение предсказаний классов для тестовой выборки
y_pred_logistic = logistic_regression_model.predict(X_test)
y_pred_decision_tree = decision_tree_model.predict(X_test)
y_pred_knn = knn_model.predict(X_test)

# Оценка точности моделей
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
accuracy_decision_tree = accuracy_score(y_test, y_pred_decision_tree)
accuracy_knn = accuracy_score(y_test, y_pred_knn)

# Оценка ROC AUC для каждой модели
roc_auc_logistic = roc_auc_score(y_test, y_prob_logistic)
roc_auc_decision_tree = roc_auc_score(y_test, y_prob_decision_tree)
roc_auc_knn = roc_auc_score(y_test, y_prob_knn)

# Оценка precision и recall для каждой модели
precision_logistic = precision_score(y_test, y_pred_logistic)
precision_decision_tree = precision_score(y_test, y_pred_decision_tree)
precision_knn = precision_score(y_test, y_pred_knn)

recall_logistic = recall_score(y_test, y_pred_logistic)
recall_decision_tree = recall_score(y_test, y_pred_decision_tree)
recall_knn = recall_score(y_test, y_pred_knn)

# Вывод результатов оценки моделей
print(f'Accuracy of Logistic Regression: {accuracy_logistic:.2f}')
print(f'Accuracy of Decision Tree: {accuracy_decision_tree:.2f}')
print(f'Accuracy of K-Nearest Neighbors: {accuracy_knn:.2f}')
print(f'ROC AUC of Logistic Regression: {roc_auc_logistic:.2f}')
print(f'ROC AUC of Decision Tree: {roc_auc_decision_tree:.2f}')
print(f'ROC AUC of K-Nearest Neighbors: {roc_auc_knn:.2f}')
print(f'Precision of Logistic Regression: {precision_logistic:.2f}')
print(f'Precision of Decision Tree: {precision_decision_tree:.2f}')
print(f'Precision of K-Nearest Neighbors: {precision_knn:.2f}')
print(f'Recall of Logistic Regression: {recall_logistic:.2f}')
print(f'Recall of Decision Tree: {recall_decision_tree:.2f}')
print(f'Recall of K-Nearest Neighbors: {recall_knn:.2f}')

# Визуализация ROC-кривой для каждого классификатора
plt.figure(figsize=(10, 6))
for model_name, y_prob in zip(['Logistic Regression', 'Decision Tree', 'K-Nearest Neighbors'],
                               [y_prob_logistic, y_prob_decision_tree, y_prob_knn]):
    fpr, tpr, _ = roc_curve(y_test, y_prob)  # Получение значений FPR и TPR
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc_score(y_test, y_prob):.2f})')

plt.plot([0, 1], [0, 1], 'k--')  # Линия случайного классификатора
plt.xlabel('False Positive Rate')  # Подпись оси X
plt.ylabel('True Positive Rate')  # Подпись оси Y
plt.title('ROC Curve')  # Заголовок графика
plt.legend()  # Легенда
plt.show()  # Показать график

# Вывод значений AUC еще раз в конце для ясности
print(f'Final ROC AUC of Logistic Regression: {roc_auc_logistic:.2f}')
print(f'Final ROC AUC of Decision Tree: {roc_auc_decision_tree:.2f}')
print(f'Final ROC AUC of K-Nearest Neighbors: {roc_auc_knn:.2f}')

## Экспериментируйте
Для получения лучшего качества придется поэкспериментировать. Подсказка: попробуйте оптимизировать гиперпараметры модели